In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import itertools
import csv
import os
import re
from requests import get

import unicodedata
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")

# import acquire
# from acquire import get_article_text

## Changing the acquire function to take a list of lists.

In [398]:
# def get_url_list():
#     urls = []
#     headers = {'User-Agent': 'Student'}
#     url = 'https://github.com/search?o=desc&p=1&q=stars%3A%3E1&s=updated&type=Repositories'
#     response = get(url, headers=headers)
#     soup = BeautifulSoup(response.content, 'html.parser')
#     max_page = 3
#     for i in range(1,max_page):
#         url = 'https://github.com/search?o=desc&p=' + str(i) + '&q=stars%3A%3E1&s=updated&type=Repositories/'

#         response = get(url, headers=headers)
#         soup = BeautifulSoup(response.text)
          
#         repo = soup.findAll('li', class_='repo-list-item d-flex flex-column flex-md-row flex-justify-start py-4 public source')
#         for div in repo:
#             one = []
#             one.append(re.findall(r'.*/.*',div.text)[0])
#             urls.append(one)

#     final_urls = []
#     for url in urls:
#         url = 'https://github.com/'+url[0]
#         final_urls.append(url)
        
#     return final_urls

In [399]:
# url_list = get_url_list()

In [561]:
# url_list

In [559]:
# def get_article_text_with_labels():

#     blobs = list(url_list)
#     good_blobs = []
#     read_me_list = []
#     language_top = []
#     stars = []
#     for blob in blobs:
        
#         blob_string=blob.replace(' ', '')
        
#         url_1 = str(blob_string)
#         url = url_1

#         response = requests.get(url)
        
#         soup = BeautifulSoup(response.text) 
# #         print(soup)
        
#         read_me = soup.find(class_='markdown-body entry-content p-5')

#         if read_me != None:
#             if soup.find('span', class_='lang') != None:
#                 language_rm = soup.find('span', class_='lang').text
#                 language_top.append(str(language_rm))
#             else: 
#                 language_top.append('None')
                
#             if soup.find('a', class_='social-count js-social-count') != None:
#                 stars_count = soup.find('a', class_='social-count js-social-count')
#                 stars_count = stars_count.attrs['aria-label']
#                 stars.append(str(stars_count))
#             else: 
#                 stars.append('0')
                
#             read_me_list.append(read_me.text)
#             good_blobs.append(str(blob))
#             stars.append(stars_count)
            
# #     return pd.DataFrame(list(zip(good_blobs,language_top, read_me_list, stars)))         
#     df = pd.DataFrame(list(zip(good_blobs,language_top, read_me_list, stars))) 
#     df.columns = ['title', 'language', 'original', 'stars']
    
#     art_nums = list(range(df.title.count()))        
#     for art_num in art_nums:
#         df.title[art_num] = df.title[art_num].split('.com/')[1]
# #         df.title[art_num] = df.title[art_num].replace(']', '')
# #         df.title[art_num] = df.title[art_num].replace('\'', '')
    
#     return df

In [562]:
# get_article_text_with_labels()

In [2]:
df = pd.read_csv('github_data.csv', header='infer', index_col=0)
df.head(3)

,title,language,original,stars
0,IamTinashe/College-Hub,Vue,College-Hub\n\nWeb-based accommodation service...,2
1,EetuPe/Bounce,JavaScript,Bounce\nBounce (or bounce 2 : the ultimate rem...,3
2,machsix/Super-preloader,JavaScript,\n \nSuper-preloader\nDocument\nComplete D...,17


## Did something wrong happen?  There are a lot of non English characters.  A manual inspection of this readme confirms this is what's in it.  Later on, we'll run a 'basic clean'.  We'll look at the % of words dropped.  If a document returns with only 50-60% of it's original words, it might contain a lot of items that are not considered English.

## Let's work on the columns for 'word counts'.  This is a loose term as we can visually see that some items that were originally hyphenated, and originally considered a single word, come back as multiple words.  Because of this, some readme files actually return more 'words' that the original text contained.

In [3]:
def original_word_count():
    df[['original_cnt']] = df[['original']]
    art_nums = list(range(df.original.count()))
    for art_num in art_nums:
        df.original_cnt[art_num] = len(df['original'][art_num].split())
    df['original_cnt']=df['original_cnt'].astype(int)
    return df

In [4]:
original_word_count()
df.head()

,title,language,original,stars,original_cnt
0,IamTinashe/College-Hub,Vue,College-Hub\n\nWeb-based accommodation service...,2,55
1,EetuPe/Bounce,JavaScript,Bounce\nBounce (or bounce 2 : the ultimate rem...,3,42
2,machsix/Super-preloader,JavaScript,\n \nSuper-preloader\nDocument\nComplete D...,17,372
3,dotnet/docs.tr-tr,PowerShell,\n\n\nms.openlocfilehash\nms.sourcegitcommit\n...,8,150
4,vadxq/pushQQlove,JavaScript,pushQQlove\nweather and to remind of the time ...,2,309


In [5]:
def basic_clean():
    
    df[['article']] = df[['original']]
    art_nums = list(range(df.original.count()))
    for art_num in art_nums:
        original = df.article[art_num]
        article = re.sub(r'\s', ' ', original).lower()
        normalized = unicodedata.normalize('NFKD', article)
        just_ascii = normalized.encode('ascii', 'ignore')
        article = just_ascii.decode('utf-8')
        without_special_chars = re.sub(r'[^\w\s]', ' ', article)
        word_list = without_special_chars.split()
        article_final = ' '.join(word_list)
        df.article[art_num] = article_final
    return df

In [6]:
basic_clean().head()

,title,language,original,stars,original_cnt,article
0,IamTinashe/College-Hub,Vue,College-Hub\n\nWeb-based accommodation service...,2,55,college hub web based accommodation service bu...
1,EetuPe/Bounce,JavaScript,Bounce\nBounce (or bounce 2 : the ultimate rem...,3,42,bounce bounce or bounce 2 the ultimate remaste...
2,machsix/Super-preloader,JavaScript,\n \nSuper-preloader\nDocument\nComplete D...,17,372,super preloader document complete document dif...
3,dotnet/docs.tr-tr,PowerShell,\n\n\nms.openlocfilehash\nms.sourcegitcommit\n...,8,150,ms openlocfilehash ms sourcegitcommit ms trans...
4,vadxq/pushQQlove,JavaScript,pushQQlove\nweather and to remind of the time ...,2,309,pushqqlove weather and to remind of the time a...


In [7]:
def article_word_count():
    df[['article_cnt']] = df[['article']]
    art_nums = list(range(df.original.count()))
    for art_num in art_nums:
        df.article_cnt[art_num] = len(df['article'][art_num].split())
    df['article_cnt']=df['article_cnt'].astype(int)
    return df

In [8]:
article_word_count().head()

,title,language,original,stars,original_cnt,article,article_cnt
0,IamTinashe/College-Hub,Vue,College-Hub\n\nWeb-based accommodation service...,2,55,college hub web based accommodation service bu...,50
1,EetuPe/Bounce,JavaScript,Bounce\nBounce (or bounce 2 : the ultimate rem...,3,42,bounce bounce or bounce 2 the ultimate remaste...,40
2,machsix/Super-preloader,JavaScript,\n \nSuper-preloader\nDocument\nComplete D...,17,372,super preloader document complete document dif...,394
3,dotnet/docs.tr-tr,PowerShell,\n\n\nms.openlocfilehash\nms.sourcegitcommit\n...,8,150,ms openlocfilehash ms sourcegitcommit ms trans...,159
4,vadxq/pushQQlove,JavaScript,pushQQlove\nweather and to remind of the time ...,2,309,pushqqlove weather and to remind of the time a...,189


In [9]:
def article_percent():    
    df[['article_per_kept']] = df[['article_cnt']]
    df[['article_per_kept']] = df.article_cnt / df.original_cnt
    df['article_per_kept']=df['article_per_kept'].astype(float)
    return df

In [10]:
article_percent()

,title,language,original,stars,original_cnt,article,article_cnt,article_per_kept
0,IamTinashe/College-Hub,Vue,College-Hub\n\nWeb-based accommodation service...,2,55,college hub web based accommodation service bu...,50,0.909091
1,EetuPe/Bounce,JavaScript,Bounce\nBounce (or bounce 2 : the ultimate rem...,3,42,bounce bounce or bounce 2 the ultimate remaste...,40,0.952381
2,machsix/Super-preloader,JavaScript,\n \nSuper-preloader\nDocument\nComplete D...,17,372,super preloader document complete document dif...,394,1.059140
3,dotnet/docs.tr-tr,PowerShell,\n\n\nms.openlocfilehash\nms.sourcegitcommit\n...,8,150,ms openlocfilehash ms sourcegitcommit ms trans...,159,1.060000
4,vadxq/pushQQlove,JavaScript,pushQQlove\nweather and to remind of the time ...,2,309,pushqqlove weather and to remind of the time a...,189,0.611650
5,bajumar/Azure-Monitor-Logs,None,My Azure Monitor Logs Repo\nThis repo contains...,2,98,my azure monitor logs repo this repo contains ...,96,0.979592
6,gigantum/gigantum-testing,Python,Gigantum Testing\nAutomation of Gigantum testi...,3,218,gigantum testing automation of gigantum testin...,208,0.954128
7,dequelabs/cauldron-react,JavaScript,Cauldron React\nThis project is used internall...,4,297,cauldron react this project is used internally...,298,1.003367
8,c-koi/gmic-qt,C++,G'MIC-Qt: a versatile G'MIC plugin\nPurpose\nG...,44,249,g mic qt a versatile g mic plugin purpose g mi...,315,1.265060
9,GeneZharov/introversion,JavaScript,Introversion.js\nTool for debugging JavaScript...,8,3192,introversion js tool for debugging javascript ...,3075,0.963346


## Let's take a look at files that returned less than 70% of their original counts.  They do appear to have a lot of non-English characters as suspected.

In [12]:
df[df.article_per_kept < .7]

,title,language,original,stars,original_cnt,article,article_cnt,article_per_kept
4,vadxq/pushQQlove,JavaScript,pushQQlove\nweather and to remind of the time ...,2,309,pushqqlove weather and to remind of the time a...,189,0.611650
43,dotnet/docs.zh-tw,PowerShell,.NET Docs\n此存放庫包含 .NET 的概念文件。 .NET 文件網站使用了多個存放...,21,44,net docs net net api net compiler platform sdk...,22,0.500000
49,dotnet/docs.zh-tw,PowerShell,.NET Docs\n此存放庫包含 .NET 的概念文件。 .NET 文件網站使用了多個存放...,21,44,net docs net net api net compiler platform sdk...,22,0.500000
57,dotnet/docs.zh-tw,PowerShell,.NET Docs\n此存放庫包含 .NET 的概念文件。 .NET 文件網站使用了多個存放...,21,44,net docs net net api net compiler platform sdk...,22,0.500000
62,dotnet/docs.zh-tw,PowerShell,.NET Docs\n此存放庫包含 .NET 的概念文件。 .NET 文件網站使用了多個存放...,21,44,net docs net net api net compiler platform sdk...,22,0.500000
98,vadxq/pushQQlove,JavaScript,pushQQlove\nweather and to remind of the time ...,2,309,pushqqlove weather and to remind of the time a...,189,0.611650
101,vadxq/pushQQlove,JavaScript,pushQQlove\nweather and to remind of the time ...,2,309,pushqqlove weather and to remind of the time a...,189,0.611650
123,dotnet/docs.zh-cn,PowerShell,.NET Docs\n此存储库包含适用于 .NET 的概念文档。 .NET 文档站点建立在多...,265,34,net docs net net api net compiler platform sdk...,14,0.411765
130,dotnet/docs.ja-jp,PowerShell,.NET ドキュメント\nこのリポジトリには、.NET の概念に関するドキュメントが含まれて...,28,40,net net net api net compiler platform sdk 3 ne...,20,0.500000
136,dotnet/docs.ja-jp,PowerShell,.NET ドキュメント\nこのリポジトリには、.NET の概念に関するドキュメントが含まれて...,28,40,net net net api net compiler platform sdk 3 ne...,20,0.500000


In [13]:
def lemmatize():
    
    df[['lemmatized']] = df[['article']]
    art_nums = list(range(df.article.count()))
    wnl = nltk.stem.WordNetLemmatizer()
    for art_num in art_nums:
        lemm_article = df.lemmatized[art_num]
        lemmas = [wnl.lemmatize(word) for word in lemm_article.split()]
        article_lemmatized = ' '.join(lemmas)
        df.lemmatized[art_num] = article_lemmatized
    return df

In [14]:
lemmatize().head()

,title,language,original,stars,original_cnt,article,article_cnt,article_per_kept,lemmatized
0,IamTinashe/College-Hub,Vue,College-Hub\n\nWeb-based accommodation service...,2,55,college hub web based accommodation service bu...,50,0.909091,college hub web based accommodation service bu...
1,EetuPe/Bounce,JavaScript,Bounce\nBounce (or bounce 2 : the ultimate rem...,3,42,bounce bounce or bounce 2 the ultimate remaste...,40,0.952381,bounce bounce or bounce 2 the ultimate remaste...
2,machsix/Super-preloader,JavaScript,\n \nSuper-preloader\nDocument\nComplete D...,17,372,super preloader document complete document dif...,394,1.059140,super preloader document complete document dif...
3,dotnet/docs.tr-tr,PowerShell,\n\n\nms.openlocfilehash\nms.sourcegitcommit\n...,8,150,ms openlocfilehash ms sourcegitcommit ms trans...,159,1.060000,m openlocfilehash m sourcegitcommit m translat...
4,vadxq/pushQQlove,JavaScript,pushQQlove\nweather and to remind of the time ...,2,309,pushqqlove weather and to remind of the time a...,189,0.611650,pushqqlove weather and to remind of the time a...


In [15]:
def lemm_word_count():
    df[['lemm_cnt']] = df[['article']]
    art_nums = list(range(df.original.count()))
    for art_num in art_nums:
        df.lemm_cnt[art_num] = len(df['lemmatized'][art_num].split())
        
    df.lemm_cnt = df.lemm_cnt.astype(int)
    return df

In [16]:
lemm_word_count().head()

,title,language,original,stars,original_cnt,article,article_cnt,article_per_kept,lemmatized,lemm_cnt
0,IamTinashe/College-Hub,Vue,College-Hub\n\nWeb-based accommodation service...,2,55,college hub web based accommodation service bu...,50,0.909091,college hub web based accommodation service bu...,50
1,EetuPe/Bounce,JavaScript,Bounce\nBounce (or bounce 2 : the ultimate rem...,3,42,bounce bounce or bounce 2 the ultimate remaste...,40,0.952381,bounce bounce or bounce 2 the ultimate remaste...,40
2,machsix/Super-preloader,JavaScript,\n \nSuper-preloader\nDocument\nComplete D...,17,372,super preloader document complete document dif...,394,1.059140,super preloader document complete document dif...,394
3,dotnet/docs.tr-tr,PowerShell,\n\n\nms.openlocfilehash\nms.sourcegitcommit\n...,8,150,ms openlocfilehash ms sourcegitcommit ms trans...,159,1.060000,m openlocfilehash m sourcegitcommit m translat...,159
4,vadxq/pushQQlove,JavaScript,pushQQlove\nweather and to remind of the time ...,2,309,pushqqlove weather and to remind of the time a...,189,0.611650,pushqqlove weather and to remind of the time a...,189


## The number of words stays the same for each file after lemmatization.  Some of the words have changed, CSS is now just CS.  

In [17]:
df[df.article_cnt != df.lemm_cnt]

,title,language,original,stars,original_cnt,article,article_cnt,article_per_kept,lemmatized,lemm_cnt


## Let's remove stopwords and create two more columns, one for the count of words after stopwords removed and another than shows the percentage of words still left.

In [20]:
def remove_stopwards(extra_words=None, exclude_words=None):
    df = lemmatize()
    stopwords = nltk.corpus.stopwords.words('english')

    
    if extra_words != None:
        stopwords = stopwords + extra_words
    if exclude_words != None:
        stopwords = [word for word in stopwords if word not in exclude_words]
        
    
    df[['clean']] = df[['lemmatized']]
    df[['word_cnt_after_stop']] = df[['title']]

    art_nums = list(range(df.clean.count()))
    for art_num in art_nums:
        words = df.clean[art_num].split()
        without_stopwords = [word for word in words if word not in stopwords]
        df.word_cnt_after_stop[art_num] = len(without_stopwords)
        without_stopwords_join = ' '.join(without_stopwords)
        df.clean[art_num] = without_stopwords_join
    df[['per_stopwords_kept']] = df[['title']]
    df[['per_stopwords_kept']] = df.word_cnt_after_stop / df.lemm_cnt
    df[['per_stopwords_kept']] = df[['per_stopwords_kept']].astype(float)
    
    df[['per_overall_kept']] = df[['title']]
    df[['per_overall_kept']] = df.word_cnt_after_stop / df.original_cnt
    df[['per_overall_kept']] = df[['per_overall_kept']].astype(float)
   
    return df

In [21]:
remove_stopwards().head()

,title,language,original,stars,original_cnt,article,article_cnt,article_per_kept,lemmatized,lemm_cnt,clean,word_cnt_after_stop,per_stopwords_kept
0,IamTinashe/College-Hub,Vue,College-Hub\n\nWeb-based accommodation service...,2,55,college hub web based accommodation service bu...,50,0.909091,college hub web based accommodation service bu...,50,college hub web based accommodation service bu...,43,0.860000
1,EetuPe/Bounce,JavaScript,Bounce\nBounce (or bounce 2 : the ultimate rem...,3,42,bounce bounce or bounce 2 the ultimate remaste...,40,0.952381,bounce bounce or bounce 2 the ultimate remaste...,40,bounce bounce bounce 2 ultimate remaster wish ...,26,0.650000
2,machsix/Super-preloader,JavaScript,\n \nSuper-preloader\nDocument\nComplete D...,17,372,super preloader document complete document dif...,394,1.059140,super preloader document complete document dif...,394,super preloader document complete document dif...,245,0.621827
3,dotnet/docs.tr-tr,PowerShell,\n\n\nms.openlocfilehash\nms.sourcegitcommit\n...,8,150,ms openlocfilehash ms sourcegitcommit ms trans...,159,1.060000,m openlocfilehash m sourcegitcommit m translat...,159,openlocfilehash sourcegitcommit translationtyp...,148,0.930818
4,vadxq/pushQQlove,JavaScript,pushQQlove\nweather and to remind of the time ...,2,309,pushqqlove weather and to remind of the time a...,189,0.611650,pushqqlove weather and to remind of the time a...,189,pushqqlove weather remind time others dev npm ...,175,0.925926


In [22]:
df.describe()

,stars,original_cnt,article_cnt,article_per_kept,lemm_cnt,per_stopwords_kept
count,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000
mean,600.509677,450.206452,474.270968,1.106672,474.270968,0.730738
std,1793.949294,633.013302,657.000794,0.439207,657.000794,0.127427
min,2.000000,1.000000,3.000000,0.411765,3.000000,0.468504
25%,3.000000,98.000000,101.000000,1.000000,101.000000,0.646688
50%,12.000000,233.000000,230.000000,1.050459,230.000000,0.706897
75%,84.000000,415.000000,454.000000,1.083089,454.000000,0.785991
max,8910.000000,3192.000000,3417.000000,3.600000,3417.000000,1.000000


In [24]:
df[df.per_stopwords_kept < .6]

,title,language,original,stars,original_cnt,article,article_cnt,article_per_kept,lemmatized,lemm_cnt,clean,word_cnt_after_stop,per_stopwords_kept
5,bajumar/Azure-Monitor-Logs,None,My Azure Monitor Logs Repo\nThis repo contains...,2,98,my azure monitor logs repo this repo contains ...,96,0.979592,my azure monitor log repo this repo contains e...,96,azure monitor log repo repo contains everythin...,50,0.520833
11,damng/hackernews-rss-with-inlined-content,Python,hackernews-rss-inlined-content\nLoads the hack...,10,176,hackernews rss inlined content loads the hacke...,202,1.147727,hackernews r inlined content load the hackerne...,202,hackernews r inlined content load hackerness r...,120,0.594059
24,artsy/palette,TypeScript,@artsy/palette \nArtsy's Design System\nMeta...,62,277,artsy palette artsy s design system meta point...,294,1.061372,artsy palette artsy s design system meta point...,294,artsy palette artsy design system meta point p...,175,0.595238
31,artsy/palette,TypeScript,@artsy/palette \nArtsy's Design System\nMeta...,62,277,artsy palette artsy s design system meta point...,294,1.061372,artsy palette artsy s design system meta point...,294,artsy palette artsy design system meta point p...,175,0.595238
67,microsoft/clrmd,C#,Microsoft.Diagnostics.Runtime\n\nMicrosoft.Dia...,515,176,microsoft diagnostics runtime microsoft diagno...,176,1.000000,microsoft diagnostics runtime microsoft diagno...,176,microsoft diagnostics runtime microsoft diagno...,99,0.562500
70,nss-day-cohort-30/bangazon-api-whimsical-whoop...,C#,"Building the Bangazon Platform API\nWelcome, n...",3,382,building the bangazon platform api welcome new...,384,1.005236,building the bangazon platform api welcome new...,384,building bangazon platform api welcome new ban...,216,0.562500
99,bajumar/Azure-Monitor-Logs,None,My Azure Monitor Logs Repo\nThis repo contains...,2,98,my azure monitor logs repo this repo contains ...,96,0.979592,my azure monitor log repo this repo contains e...,96,azure monitor log repo repo contains everythin...,50,0.520833
102,bajumar/Azure-Monitor-Logs,None,My Azure Monitor Logs Repo\nThis repo contains...,2,98,my azure monitor logs repo this repo contains ...,96,0.979592,my azure monitor log repo this repo contains e...,96,azure monitor log repo repo contains everythin...,50,0.520833
108,dOrgTech/DAOcreator,TypeScript,DAOcreator\nWizard for setting up your own DAO...,7,41,daocreator wizard for setting up your own daos...,43,1.048780,daocreator wizard for setting up your own daos...,43,daocreator wizard setting daostack dao interes...,23,0.534884
113,dOrgTech/DAOcreator,TypeScript,DAOcreator\nWizard for setting up your own DAO...,7,41,daocreator wizard for setting up your own daos...,43,1.048780,daocreator wizard for setting up your own daos...,43,daocreator wizard setting daostack dao interes...,23,0.534884
